# 텍스트 데이터를 이용한 텍스트 데이터 로드 및 전처리

In [52]:
import pandas as pd 
import numpy as np

clean_data = pd.read_csv('twitter-airline-sentiment.csv')
clean_data.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


## Basic Text Pre-Processing

텍스트 전처리 단계에는 사용 가능한 텍스트 데이터를 추가로 정리하기 위한 몇 가지 필수 작업이 포함됩니다. 여기에는 다음과 같은 작업이 포함됩니다.

**1. 불용어 (stopword) 제거** : 영어에서 a, an, the, as, in, on 등과 같은 단어는 불용어로 간주되므로 요구 사항에 따라 이러한 단어를 제거하여 어휘 크기를 줄일 수 있습니다. 어떤 특정한 의미를 가지고

**2. 소문자 변환 :** 대소문자가 문제에 영향을 미치지 않을 수 있으므로 모든 단어를 소문자로 변환합니다. 그렇게 함으로써 우리는 어휘의 크기를 줄이고 있습니다.

**3. 형태소 분석 :** 형태소 분석은 접미사를 제거하고 해당 단어의 모든 다른 변형이 동일한 형태로 표현될 수 있도록 단어를 일부 기본 형태로 줄이는 과정을 말합니다(예: "walk"와 "walking"은 모두 " 걷다").

**4. 토큰화 :** NLP 소프트웨어는 일반적으로 텍스트를 단어(토큰)와 문장으로 분해하여 텍스트를 분석합니다.

In [53]:
# 필요 없는 컬럼명 제거
waste_col = ['tweet_id', 'airline_sentiment_confidence',
       'negativereason', 'negativereason_confidence', 'airline',
       'airline_sentiment_gold', 'name', 'negativereason_gold',
       'retweet_count', 'tweet_coord', 'tweet_created',
       'tweet_location', 'user_timezone']

data = clean_data.drop(waste_col, axis = 1)
data.head()

,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...


In [54]:
# nltk 라이브러리를 이용해 불용어, 소문자 변환, 형태소 분석, 토큰화 진행
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer 
from nltk.tokenize import RegexpTokenizer

# preprocess_text라는 함수를 생성하여 문장이 들어오면 
# 불용어 처리, 소문자 변환, 형태소 분석, 토큰화 진행를 하는 함수를 생성하세요.

def preprocess_text(sentence):

    # 불용어(stopwords) 처리를 위해 영어 불용어 리스트를 불러옵니다.
    stop_words = set(stopwords.words('english'))
    
    # 문장을 소문자로 변환합니다.
    sentence = sentence.lower()
    
    # 정규 표현식을 이용해 문장을 토큰화합니다.
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(sentence)
    
    # 형태소 분석을 위해 스노우볼 스태머 객체를 생성합니다.
    stemmer = SnowballStemmer('english')
    
    # 토큰 중에서 불용어가 아닌 단어에 대해 형태소 분석을 하고, 스태머를 적용합니다.
    tokens = [stemmer.stem(word) for word in tokens if word not in stop_words]
    
    # 처리된 토큰 리스트를 반환합니다.
    return tokens

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ebdl\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [55]:
# preprocess_text 생성하고 출력 테스트

print(f"Orignal Text : {data.text[0]}")
print()
print(f"Preprocessed Text : {preprocess_text(data.text[0])}")

# 기존 data에 preprocess_text를 적용하여 모든 문장을 전처리
preprocessed_data = []

for sentence in data.text:
    preprocessed_sentence = preprocess_text(sentence)
    preprocessed_data.append(preprocessed_sentence)

Orignal Text : @VirginAmerica What @dhepburn said.

Preprocessed Text : ['virginamerica', 'dhepburn', 'said']


## One-hot Encoding

원-핫 인코딩에서 말뭉치 어휘의 각 단어 w에는 1과 |V| 사이의 고유한 정수 ID(wid)가 지정되며, 여기서 V는 말뭉치 어휘 집합입니다. 그런 다음 각 단어는 0과 1의 V 차원 이진 벡터로 표시됩니다. 이것은 |V| 인덱스를 제외하고 모두 0으로 채워진 차원 벡터입니다. 여기서 index = wid입니다. 이 인덱스에서 우리는 단순히 1을 넣습니다. 그런 다음 개별 단어에 대한 표현이 결합되어 문장 표현을 형성합니다.

예시: 
One Hot Representation for sentence "the cat sat on the mat" :

![](https://miro.medium.com/max/886/1*_da_YknoUuryRheNS-SYWQ.png)

In [56]:
# 위의 예시를 이용한 vocab
sample_vocab = ['the', 'cat', 'sat', 'on', 'mat', 'dog', 'run', 'green', 'tree']

# 현재 전처리된 데이터셋을 가지고 data_vocab에 해당하는 vocab 셋 생성
data_vocab = []

# 데이터셋에 있는 모든 단어들을 하나의 리스트로 만듭니다.
all_words = [word for sentence in preprocessed_data for word in sentence]

data_vocab = set(all_words)
data_vocab =list(data_vocab)

In [65]:
# one-hot 인코딩을 실행하는 아래 함수 생성
def get_onehot_representation(text, vocab = data_vocab):
    onehot_encoded = []
    for word in text:
        onehot_encoded.append([1 if word == v else 0 for v in vocab])
    return onehot_encoded

In [66]:
# 아래 코드를 위의 예시 처럼 생성 되는지 출력 테스트
print("One Hot Representation for sentence \"the cat sat on the mat\" :")
get_onehot_representation(['the', 'cat', 'sat', 'on', 'the', 'mat'], sample_vocab)

One Hot Representation for sentence "the cat sat on the mat" :


[[1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0]]

In [67]:
# 아래 코드를 출력
print(f'Length of Vocabulary : {len(data_vocab)}')
print(f'Sample of Vocabulary : {data_vocab[302 : 312]}')

Length of Vocabulary : 12256
Sample of Vocabulary : ['l0i8fnz3ku', 'clincher', 'investig', 'sugafli', 'decent', '85832', 'angryairtravel', 'gate', 'obscen', 'unfamiliar']


## Bag-of-Words (Binary Encoding, Count-based Encoding)

BoW(Bag of Words)는 NLP, 특히 텍스트 분류 문제에서 일반적으로 사용된 고전적인 텍스트 표현 기술입니다. 핵심 아이디어는 다음과 같습니다. 순서와 문맥을 무시하면서 고려 중인 텍스트를 단어 모음으로 표현합니다.

원-핫 인코딩과 유사하게 BoW는 단어를 1과 |V| 사이의 고유한 정수 ID로 매핑합니다. 말뭉치의 각 문서는 |V|의 벡터로 변환됩니다. 차원은 벡터의 i번째 구성요소에 있었습니다. i = wid는 문서에서 단어 w가 나타나는 횟수입니다. 즉, V의 각 단어를 문서에서 발생한 횟수로 점수를 매깁니다.

아래의 예시를 들어 알아보자.

단어로 구성된 어휘 V가 있다고 가정해 보겠습니다. **V --> {the, cat, sat, in, hat, with}** 그러면 몇 문장의 bag of word 표현은 다음과 같이 주어집니다.

![](https://miro.medium.com/max/1400/1*3IACMnNpwVlCl8kSTJocPA.png)

In [70]:
# sklearn의 CountVectorizer를 통해서 하나의 샘플 문장을 Bag of Words 형태로 바꿀 수 있다.
# 아래의 코드를 실행해보고 이해하여라.

from sklearn.feature_extraction.text import CountVectorizer

sample_bow = CountVectorizer()

sample_corpus = ["the cat sat", "the cat sat in the hat", "the cat with the hat"]

sample_bow.fit(sample_corpus)

def get_bow_representation(text):
        return sample_bow.transform(text)
    
print(f"Vocabulary mapping for given sample corpus : \n {sample_bow.vocabulary_}")
print("\nBag of word Representation of sentence 'the cat cat sat in the hat'")
print(get_bow_representation(["the cat cat sat in the hat"]).toarray())

Vocabulary mapping for given sample corpus : 
 {'the': 4, 'cat': 0, 'sat': 3, 'in': 2, 'hat': 1, 'with': 5}

Bag of word Representation of sentence 'the cat cat sat in the hat'
[[2 1 1 1 2 0]]


In [72]:
# 현재 전처리된 데이터셋을 가지고 data의 text 부분을 
# bag-of-words 형태로 변환하고 해당 데이터프레임에 저장하고 일부를 출력해보아라.

# CountVectorizer 객체 생성
bow = CountVectorizer()

# 데이터셋에서 text 부분 추출
texts = data_vocab

# bag-of-words 형태로 변환
bow_repr = bow.fit_transform(texts)

# 변환된 결과를 데이터프레임에 저장
bow_df = pd.DataFrame(bow_repr.toarray(), columns=bow.get_feature_names())

# 일부 출력
print(bow_df.head())

   00  000  000114  000419  000ft  000lbs  0011  0016  00a  00am  ...  \
0   0    0       0       0      0       0     0     0    0     0  ...   
1   0    0       0       0      0       0     0     0    0     0  ...   
2   0    0       0       0      0       0     0     0    0     0  ...   
3   0    0       0       0      0       0     0     0    0     0  ...   
4   0    0       0       0      0       0     0     0    0     0  ...   

   zrh_airport  zsdgzydnd  zsuztnaijq  ztrdwv0n4l  zuke  zurich  zv2pt6trk9  \
0            0          0           0           0     0       0           0   
1            0          0           0           0     0       0           0   
2            0          0           0           0     0       0           0   
3            0          0           0           0     0       0           0   
4            0          0           0           0     0       0           0   

   zv6cfpohl5  zvfmxnuelj  zzps5ywve2  
0           0           0           0  
1     

C:\Users\ebdl\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


## Bag-of-Words의 장점과 단점
- 구현이 쉽고 빠른 장점이 있는 것 같고 어떤 단어가 들어가도 정량적인 특성을 가진 벡터로 나타낼 수 있는 장점이 있습니다.
- 단점으로는 단순히 단어의 빈도만을 계산하기 때문에 문맥, 순서등을 고려하지 않는 점과 벡터의 희소성이 높아질 우려가 있다는 점이있습니다.

### One-hot Encoding과의 유사점과 차이점은 무엇인가?
- 둘다 텍스트 데이터를 수치형 벡터로 변환하는데 사용한다는 점에서 비슷합니다.
- 차이점으로는 방식의 차이가 가장 큰데 원핫인코딩은 vocab만큼의 벡터 크기를 가지게 되고 해당 단어만 1이고 나머지는 0인 형태를 가지는 반면 BOW 모델은 단어의 출현 빈도를 벡터로 표현하고 마찬가지로 단어만큼의 차원을 가집니다.

### Bag-of-Words 방식의 장점과 단점은 무엇인가?
- 단어 간 유사도를 계산하여 더욱 정교한 표현이 가능하다
- 단어의 순서 정보가 손실되어 텍스트 분류에서 성능이 좋지 않을 수 있다 
